![openclassrooms](https://s3.eu-west-1.amazonaws.com/course.oc-static.com/courses/6204541/1+HnqdJ-5ofxiPP9HIxdNdpw.jpeg)

# Filtrez les données du dataframe

Nous allons à présent aller un peu plus loin dans la manipulation de nos prêts bancaires. Les demandes suivantes ont été formulées par le responsable du service prêt de notre établissement :
- Le taux d’endettement autorisé est de 35%. Pourriez vous me communiquer le nombre de personnes ayant dépassé ce seuil ?
- Même question, mais cette fois ci uniquement sur l’agence Parisienne
- Pour faciliter le traitement d’éventuelles futures demandes de prêts, pourriez vous ajouter une variable nommée `risque` qui nous permettrait d’identifier les clients risqués facilement
- Combien de prêts automobiles ont été accordés ? Quel est le coût total moyen de ces derniers ?
- Quel est le bénéfice mensuel total réalisé par l’agence Toulousaine ?

Vous allez devoir mettre en application l’ensemble des processus de sélection présentés ci-dessus pour répondre à ces différentes demandes.

In [2]:
import numpy as np
import pandas as pd

In [3]:
# traitement réalisés précédemment
prets = pd.read_csv('https://raw.githubusercontent.com/OpenClassrooms-Student-Center/fr-4452741-decouvrez-les-librairies-python-pour-la-data-science/main/data/prets.csv')

# calcul du taux d'endettement
prets['taux_endettement'] = round(prets['remboursement'] * 100 / prets['revenu'], 2)

# renommer taux en taux_interet
prets.rename(columns={'taux':'taux_interet'}, inplace=True)

# calculer le cout total du pret
prets['cout_total'] = prets['remboursement'] * prets['duree']

# calculer les bénéfices mensuels réalisés
prets['benefices'] = round((prets['cout_total'] * prets['taux_interet']/100)/(24), 2)

prets.head()

,identifiant,ville,CP,revenu,remboursement,duree,type,taux_interet,taux_endettement,cout_total,benefices
0,0,TOULOUSE,31100,3669.0,1130.05,240,immobilier,1.168,30.80,271212.0,131.99
1,1,PARIS,75009,5310.0,240.00,64,automobile,3.701,4.52,15360.0,23.69
2,1,PARIS,75009,5310.0,1247.85,300,immobilier,1.173,23.50,374355.0,182.97
3,2,MARSEILLE,13010,1873.0,552.54,240,immobilier,0.972,29.50,132609.6,53.71
4,3,MARSEILLE,13010,1684.0,586.03,180,immobilier,1.014,34.80,105485.4,44.57


Le taux d’endettement autorisé est de 35%. Pourriez vous me communiquer le nombre de personnes ayant dépassé ce seuil ? vous pourrez me stocker la liste dans une variable à part nommé `clients_risque`

In [ ]:
nombre_client_risque = prets.loc[prets['taux_endettement'] > 35, :].shape[0]
print('Il y a', nombre_client_risque, 'clients qui ont dépassé le seuil autorisé')

Il y a 58 clients qui ont dépassé le seuil autorisé


*Pour expliquer rapidement cette ligne de code :*
- *dans un premier temps nous utilisons la fonction .loc, pour sélectionner l'ensemble des lignes dont le taux d'endettement est supérieur à 35%*
- *nous utilisons ensuite la méthode `.shape` pour calculer le nombre de clients concernés*

Même question, mais cette fois ci uniquement sur l’agence Parisienne

In [ ]:
nombre_client_risque = prets.loc[(prets['taux_endettement'] > 35) & (prets['ville'] == 'PARIS'), :].shape[0]
print('Il y a', nombre_client_risque, "clients qui ont dépassé le seuil autorisé à l'agence Parisienne")

Il y a 16 clients qui ont dépassé le seuil autorisé à l'agence Parisienne


Pour faciliter le traitement d’éventuelles futures demandes de prêts, pourriez vous ajouter une variable nommée `risque` qui aurait pour valeur `Oui` si le client est risqué (taux d’endettement > 35%), `Non` sinon.
Pour se faire, je vous conseille de créer la variable `risque` en l'initialisant à `Non` (ou `Oui`, à votre préférence) et de ne modifier que les lignes concernées par la valeur inverse :

In [ ]:
prets['risque'] = 'Non'
prets.loc[prets['taux_endettement'] > 35, 'risque'] = 'Oui'

Combien de prêts automobiles ont été accordés ? Quel est le coût total moyen de ces derniers ?

In [ ]:
prets_auto = prets.loc[prets['type'] == 'automobile', :]
print('Nous avons accordé', prets_auto.shape[0], 'prêts automobiles, dont le coût total moyen est de',
      prets_auto['cout_total'].mean(), '€')

Nous avons accordé 16 prêts automobiles, dont le coût total moyen est de 13665.0 €


*Pour expliquer rapidement ces lignes de code :*
- *nous opérons une sélection sur les prêts étant de type automobile. Le résultat de cette sélection est stocké dans un dataframe nommé `prets_auto`*
- *nous utilisons la méthode `.shape` comme précédemment pour déterminer le nombre de prêts concernés*
- *nous calculons ensuite la moyenne de la variable `cout_total` de ce dataframe nouvellement créé*

Quel est le bénéfice mensuel total réalisé par l’agence Toulousaine ?

In [ ]:
prets.loc[prets['ville'] == 'TOULOUSE', 'benefices'].sum()

2138.0699999999997

In [11]:
prets.groupby(['ville','type'])['remboursement'].sum()

ville      type      
BORDEAUX   immobilier    15330.59
LYON       automobile      788.80
           immobilier    25275.87
MARSEILLE  automobile      839.60
           immobilier    40056.12
NICE       automobile      612.80
           immobilier    22931.44
PARIS      automobile     1426.60
           immobilier    92626.01
TOULOUSE   automobile      718.80
           immobilier    18567.38
Name: remboursement, dtype: float64

In [20]:
data_pivoted = prets.pivot_table(index='ville', columns='type', values='remboursement', aggfunc='sum')
data_pivoted

type,automobile,immobilier
ville,,
BORDEAUX,NaN,15330.59
LYON,788.8,25275.87
MARSEILLE,839.6,40056.12
NICE,612.8,22931.44
PARIS,1426.6,92626.01
TOULOUSE,718.8,18567.38
